In [19]:
import pandas as pd
import json
import os
import dask.bag as dk
import mysql.connector as db
from dask import delayed

In [23]:
# Create connection
cnx = db.connect(
    host="localhost",
    port=3306,
    user="root",
    password="root"
)

db_curr = cnx.cursor()

# Create database if it doesn't exist
db_curr.execute("CREATE DATABASE IF NOT EXISTS cricket_dataset_practice")

# Connect to the database
cnx = db.connect(
    host="localhost",
    port=3306,
    user="root",
    password="root",
    database="cricket_dataset_practice",
    autocommit=True
)

db_curr = cnx.cursor()

In [7]:
#create match_details table
db_curr.execute("""CREATE TABLE match_details (
    match_id INT PRIMARY KEY,
    city VARCHAR(255),
    gender VARCHAR(255),
    match_type VARCHAR(255),
    match_type_number INT,
    overs INT,
    season VARCHAR(10),
    team_type VARCHAR(255),
    venue VARCHAR(255),
    team1 VARCHAR(255),
    team2 VARCHAR(255),
    toss_winner VARCHAR(255),
    toss_decision VARCHAR(255),
    winner VARCHAR(255),
    outcome_type VARCHAR(255),
    outcome_value VARCHAR(255),
    player_of_match VARCHAR(255),
    balls_per_over INT,
    FOREIGN KEY (player_of_match) REFERENCES registry(person_id)            
);""")
db_curr.execute("""DESC match_details;""")
print(db_curr.fetchall())

[('match_id', 'int', 'NO', 'PRI', None, ''), ('city', 'varchar(255)', 'YES', '', None, ''), ('gender', 'varchar(255)', 'YES', '', None, ''), ('match_type', 'varchar(255)', 'YES', '', None, ''), ('match_type_number', 'int', 'YES', '', None, ''), ('overs', 'int', 'YES', '', None, ''), ('season', 'varchar(10)', 'YES', '', None, ''), ('team_type', 'varchar(255)', 'YES', '', None, ''), ('venue', 'varchar(255)', 'YES', '', None, ''), ('team1', 'varchar(255)', 'YES', '', None, ''), ('team2', 'varchar(255)', 'YES', '', None, ''), ('toss_winner', 'varchar(255)', 'YES', '', None, ''), ('toss_decision', 'varchar(255)', 'YES', '', None, ''), ('winner', 'varchar(255)', 'YES', '', None, ''), ('outcome_type', 'varchar(255)', 'YES', '', None, ''), ('outcome_value', 'varchar(255)', 'YES', '', None, ''), ('player_of_match', 'varchar(255)', 'YES', 'MUL', None, ''), ('balls_per_over', 'int', 'YES', '', None, '')]


In [8]:
db_curr.execute("""CREATE TABLE officials (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    person_id VARCHAR(255),
    official_type VARCHAR(255),
    FOREIGN KEY (match_id) REFERENCES match_details(match_id),
    FOREIGN KEY (person_id) REFERENCES registry(person_id)
);""")
db_curr.execute("""DESC officials;""")
print(db_curr.fetchall())

[('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('match_id', 'int', 'YES', 'MUL', None, ''), ('person_id', 'varchar(255)', 'YES', 'MUL', None, ''), ('official_type', 'varchar(255)', 'YES', '', None, '')]


In [6]:
db_curr.execute("""CREATE TABLE registry (
    person_id VARCHAR(255) PRIMARY KEY,
    person_name VARCHAR(255)
);""")
db_curr.execute("""DESC registry;""")
print(db_curr.fetchall())

[('person_id', 'varchar(255)', 'NO', 'PRI', None, ''), ('person_name', 'varchar(255)', 'YES', '', None, '')]


In [10]:
db_curr.execute("""CREATE TABLE players (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    person_id VARCHAR(255),
    team_name VARCHAR(255),
    FOREIGN KEY (match_id) REFERENCES match_details(match_id),
    FOREIGN KEY (person_id) REFERENCES registry(person_id)
);""")
db_curr.execute("""DESC players;""")
print(db_curr.fetchall())

[('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('match_id', 'int', 'YES', 'MUL', None, ''), ('person_id', 'varchar(255)', 'YES', 'MUL', None, ''), ('team_name', 'varchar(255)', 'YES', '', None, '')]


In [12]:
db_curr.execute("""CREATE TABLE deliveries (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,  
    innings INT,
    team VARCHAR(255),
    overs INT,
    balls INT,
    batter VARCHAR(255),
    bowler VARCHAR(255),
    non_striker VARCHAR(255),
    runs_batter INT,
    runs_extras INT,
    runs_total INT,
    powerplayed VARCHAR(255),
    powerplayed_type VARCHAR(255),
    player_out VARCHAR(255),
    dismissal_kind VARCHAR(255),
    fielders_involved VARCHAR(255),
    FOREIGN KEY (match_id) REFERENCES match_details(match_id),  -- Foreign key to match_details
    FOREIGN KEY (batter) REFERENCES registry(person_id),       -- Foreign key to registry for batter
    FOREIGN KEY (bowler) REFERENCES registry(person_id),       -- Foreign key to registry for bowler
    FOREIGN KEY (non_striker) REFERENCES registry(person_id),  -- Foreign key to registry for non-striker
    FOREIGN KEY (player_out) REFERENCES registry(person_id),    -- Foreign key to registry for player_out
    FOREIGN KEY (fielders_involved) REFERENCES registry(person_id)    -- Foreign key to registry for fielders_involved
);""")
db_curr.execute("""DESC deliveries;""")
print(db_curr.fetchall())

[('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('match_id', 'int', 'YES', 'MUL', None, ''), ('innings', 'int', 'YES', '', None, ''), ('team', 'varchar(255)', 'YES', '', None, ''), ('overs', 'int', 'YES', '', None, ''), ('balls', 'int', 'YES', '', None, ''), ('batter', 'varchar(255)', 'YES', 'MUL', None, ''), ('bowler', 'varchar(255)', 'YES', 'MUL', None, ''), ('non_striker', 'varchar(255)', 'YES', 'MUL', None, ''), ('runs_batter', 'int', 'YES', '', None, ''), ('runs_extras', 'int', 'YES', '', None, ''), ('runs_total', 'int', 'YES', '', None, ''), ('powerplayed', 'varchar(255)', 'YES', '', None, ''), ('powerplayed_type', 'varchar(255)', 'YES', '', None, ''), ('player_out', 'varchar(255)', 'YES', 'MUL', None, ''), ('dismissal_kind', 'varchar(255)', 'YES', '', None, ''), ('fielders_involved', 'varchar(255)', 'YES', 'MUL', None, '')]


In [ ]:
def insert_dataframe_DB(df_match_details,df_officials,df_registry,df_players,df_deliveries):
    # Create a new MySQL connection
    connection = db.connect(
        host='localhost',
        database='your_database',
        user='your_user',
        password='your_password'
    )

    try:
        cursor = connection.cursor()
        # Process data and insert into database
        # ...
        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Error processing {matchID}: {e}")
    

In [16]:
def extract_match_data(matchID, data):
    d = data['info']
    registry = d['registry']['people']
    
    # Create a dictionary to store the match details
    match = {
        "match_id": matchID, 
        "city": d['city'], 
        "gender": d['gender'],
        "match_type": d["match_type"], 
        "match_type_number": d["match_type_number"],
        "overs": d["overs"], 
        "season": d["season"], 
        "team_type": d["team_type"],
        "venue": d["venue"], 
        "team1": d["teams"][0], 
        "team2": d["teams"][1],
        "toss_winner": d["toss"]["winner"], 
        "toss_decision": d["toss"]["decision"],
        "winner": d["outcome"].get("winner"),
        "outcome_type": next(iter(d["outcome"].get("by", {})), d["outcome"].get("result")),
        "outcome_value": d["outcome"].get("by"),
        "player_of_match": ', '.join(registry.get(p) for p in d.get('player_of_match', [])),
        "balls_per_over": d["balls_per_over"]
    }
    
    # Create a DataFrame for the match details
    df_match_details = pd.DataFrame([match])
    df_match_details.to_sql('match_details', cnx, if_exists='append', index=False)
    
    
    # Create DataFrames for officials, registry, and players
    df_officials = pd.DataFrame([
        {"match_id": matchID, "person_id": registry[person], "official_type": off}
        for off in d['officials'] for person in d['officials'][off]
    ])
    
    df_registry = pd.DataFrame({
        "person_id": list(registry.values()),
        "person_name": list(registry.keys())
    }).drop_duplicates()
    
    df_players = pd.DataFrame([
        {"match_id": matchID, "person_id": registry[player], "team_name": team}
        for team, players in d['players'].items() for player in players
    ])

    df_officials.to_sql('officials', cnx, if_exists='append', index=False)
    df_registry.to_sql('registry', cnx, if_exists='append', index=False)
    df_players.to_sql('players', cnx, if_exists='append', index=False)
    

    # Create a list to store the deliveries
    deliveries = []
    
    # Iterate over the innings
    for inning_count, inning in enumerate(data['innings'], start=1):
        team = inning['team']
        powerplays = {pp['type']: (int(pp['from']), int(pp['to'])) for pp in inning.get('powerplays', [])}
        miscounted = {int(k): v['balls'] for k, v in inning.get('miscounted_overs', {}).items()}
        
        # Iterate over the overs
        for over in inning['overs']:
            over_num = over['over']
            balls_limit = miscounted.get(over['over'], 6)
            
            # Iterate over the deliveries
            for ball_count, delivery in enumerate(over['deliveries'], start=1):
                if ball_count > balls_limit: continue
                
                # Get the powerplay type
                powerplay_type = next((t for t, (s, e) in powerplays.items() if s <= over_num <= e), 'none')
                
                # Get the wickets
                wickets = delivery.get('wickets', [{}])[0]
                
                # Append the delivery to the list
                deliveries.append({
                    'innings': inning_count, 
                    'team': team, 
                    'overs': over_num, 
                    'balls': ball_count,
                    'batter': registry.get(delivery['batter']), 
                    'bowler': registry.get(delivery['bowler']),
                    'non_striker': registry.get(delivery['non_striker']),
                    'runs_batter': delivery['runs']['batter'], 
                    'runs_extras': delivery['runs']['extras'],
                    'runs_total': delivery['runs']['total'],
                    'powerplayed': 'yes' if powerplay_type != 'none' else 'no',
                    'powerplayed_type': powerplay_type,
                    'player_out': registry.get(wickets.get('player_out')),
                    'dismissal_kind': wickets.get('kind'),
                    'fielders_involved': ', '.join(registry.get(f['name']) for f in wickets.get('fielders', [])) if wickets.get('fielders') else None
                })
    
    # Create a DataFrame for the deliveries
    df_deliveries = pd.DataFrame(deliveries)
    df_deliveries.to_sql('deliveries', cnx, if_exists='append', index=False)

    


In [17]:
def process_json_file(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            match_ID = os.path.basename(filepath).rsplit('.', 1)[0]  # Extract match_ID
            # Process data and return a dictionary or suitable format
            extract_match_data(match_ID,data) # Call your function to process the data
            return match_ID
    except Exception as e:
        print(f"Error processing {filepath}: {e}")
        return None

In [22]:
@delayed
def get_json_files(format_path):
    files = []
    if os.path.isdir(format_path):
        for file in os.listdir(format_path):
            if file.endswith(".json"):
                filepath = os.path.join(format_path, file)
                files.append(filepath)
    return files

def process_json_files_parallel(matches_path, formats):
    all_filepaths = []

    for format_name in formats:
        format_path = os.path.join(matches_path, format_name)
        files = get_json_files(format_path)
        all_filepaths.append(files)

    all_filepaths = dk.compute(*all_filepaths)

    bags = []
    for filepaths in all_filepaths:
        bags.append(dk.from_sequence(filepaths))
    combined_bag = dk.concat(bags)
    results = combined_bag.map(process_json_file).compute()
    print(len(results))



# Example usage (same as before):
matches_path = os.path.join("matches")
formats = ["odis_json", "t20s_json", "tests_json"]
process_json_files_parallel(matches_path, formats) # assign all the dataframes.


TypeError: cannot pickle '_mysql_connector.MySQL' object

In [ ]:
#match_details, registry, officials, players, deliveries
db_curr.execute("""TRUNCATE TABLE ;""")
print(db_curr.fetchall())